### Import thư viện

In [6]:
import numpy as np
import pandas as pd
import time
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

### Đọc dữ liệu từ file

In [7]:
X_train = pd.read_csv("./2a/X_train_2a.csv")
Y_train = pd.read_csv("./2a/Y_train_2a.csv")
X_test = pd.read_csv("./2a/X_test_2a.csv")

X_train = X_train[['Size', 'Number', 'Thickness']]
Y_train = Y_train['Lung Cancer']
X_test = X_test[['Size', 'Number', 'Thickness']]

In [8]:
print(X_train)
print(Y_train)

          Size   Number Thickness
0     ex-large  scale-2      grey
1       median  scale-3      grey
2        small  scale-1      grey
3     ex-large  scale-4     light
4        small  scale-4      dark
...        ...      ...       ...
1973  ex-large  scale-1     light
1974    median  scale-2     light
1975  ex-large  scale-3      grey
1976    median  scale-4      dark
1977     large  scale-3      dark

[1978 rows x 3 columns]
0        stage-3
1        stage-1
2       negative
3        stage-1
4        stage-3
          ...   
1973     stage-3
1974    negative
1975    negative
1976     stage-1
1977     stage-1
Name: Lung Cancer, Length: 1978, dtype: object


### Tiền xử lý dữ liệu

In [9]:
# Mapping các giá trị tương ứng với số
size_mapping = {value: idx for idx, value in enumerate(X_train['Size'].unique())}
number_mapping = {value: idx for idx, value in enumerate(X_train['Number'].unique())}
thickness_mapping = {value: idx for idx, value in enumerate(X_train['Thickness'].unique())}
lung_cancer_mapping = {value: idx for idx, value in enumerate(Y_train.unique())}

In [10]:
print(size_mapping)
print(number_mapping)
print(thickness_mapping)
print(lung_cancer_mapping)

{'ex-large': 0, 'median': 1, 'small': 2, 'none': 3, 'large': 4}
{'scale-2': 0, 'scale-3': 1, 'scale-1': 2, 'scale-4': 3, 'scale-5': 4}
{'grey': 0, 'light': 1, 'dark': 2}
{'stage-3': 0, 'stage-1': 1, 'negative': 2, 'stage-2': 3}


In [11]:
# Chuẩn hóa dữ liệu chuỗi thành số
X_train['Size'] = X_train['Size'].map(size_mapping)
X_train['Number'] = X_train['Number'].map(number_mapping)
X_train['Thickness'] = X_train['Thickness'].map(thickness_mapping)

Y_train = Y_train.map(lung_cancer_mapping)

X_test['Size'] = X_test['Size'].map(size_mapping)
X_test['Number'] = X_test['Number'].map(number_mapping)
X_test['Thickness'] = X_test['Thickness'].map(thickness_mapping)

In [12]:
print(X_train)
print(Y_train)

      Size  Number  Thickness
0        0       0          0
1        1       1          0
2        2       2          0
3        0       3          1
4        2       3          2
...    ...     ...        ...
1973     0       2          1
1974     1       0          1
1975     0       1          0
1976     1       3          2
1977     4       1          2

[1978 rows x 3 columns]
0       0
1       1
2       2
3       1
4       0
       ..
1973    0
1974    2
1975    2
1976    1
1977    1
Name: Lung Cancer, Length: 1978, dtype: int64


### Áp dụng thuật toán Naive Bayes

#### P(y) = count(y) / count(Y)

In [13]:
# Tính toán prior probability (tiền xác suất), đây là xác xuất của các lớp mà không phụ thuộc vào dữ liệu X_train.
# Giả sử: tổng số mẫu stage-1 trong lớp lung cancer là 10, mà tổng số mẫu của lung cancer là 100 thì prior probability của stage-1 là 10/100 = 0.1
time_start = time.time()
prior = {}
for i in lung_cancer_mapping.values():
    prior[i] = len(Y_train[Y_train == i]) / len(Y_train)

#### P(xi|y) = count(xi, y) / count(y)

In [14]:
# Tính toán conditional probability (xác suất có điều kiện), đây là xác suất của các feature mà phụ thuộc vào dữ liệu train.
conditional = {}
for i in lung_cancer_mapping.values():
    conditional[i] = {}
    for j in X_train.columns: # nhãn của X
        conditional[i][j] = {}
        for k in X_train[j].unique(): # đặc trưng của nhãn X
            conditional[i][j][k] = len(X_train[(X_train[j] == k) & (Y_train == i)]) / len(Y_train[Y_train == i])

#### S(y) = P(y) * P(x1|y) * P(x2|y) * P(x3|y)

In [15]:
# Dự đoán dữ liệu Y dựa vào dữ liệu X_test
Y_test = []

for i in range(len(X_test)):
    probabilities = [] # lưu trữ xác suất của mỗi lớp
    for j in lung_cancer_mapping.values():
        probability = prior[j]
        for k in X_test.columns:
            probability *= conditional[j][k][X_test[k][i]]
        probabilities.append(probability)
    Y_test.append(np.argmax(probabilities))

time_end = time.time()

In [16]:
# In kết quả của conditional probability
# for i in conditional:
#     print(f"Class {i}")
#     for j in conditional[i]:
#         print(f"Feature {j}")
#         for k in conditional[i][j]:
#             print(f"Value {k}: {conditional[i][j][k]}")

In [17]:
print(Y_test)

# print the predicted classes
Y_test_arr = [list(lung_cancer_mapping.keys())[i] for i in Y_test]
Y_test_df = pd.DataFrame(Y_test_arr, columns=['Lung Cancer'])

print(Y_test_arr)
print(Y_test_df)

[3, 1, 3, 0, 3, 1, 3, 3, 2, 3]
['stage-2', 'stage-1', 'stage-2', 'stage-3', 'stage-2', 'stage-1', 'stage-2', 'stage-2', 'negative', 'stage-2']
  Lung Cancer
0     stage-2
1     stage-1
2     stage-2
3     stage-3
4     stage-2
5     stage-1
6     stage-2
7     stage-2
8    negative
9     stage-2


### Sử dụng thư viên Sklearn

In [19]:
# import sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder

In [20]:
X_train = pd.read_csv("./2a/X_train_2a.csv")
Y_train = pd.read_csv("./2a/Y_train_2a.csv")
X_test = pd.read_csv("./2a/X_test_2a.csv")

X_train = X_train[['Size', 'Number', 'Thickness']]
Y_train = Y_train['Lung Cancer']
X_test = X_test[['Size', 'Number', 'Thickness']]

In [21]:
# Mã hóa dữ liệu chuỗi thành số
class_le = LabelEncoder()
X_train['Size'] = class_le.fit_transform(X_train['Size'].values)
X_train['Number'] = class_le.fit_transform(X_train['Number'].values)
X_train['Thickness'] = class_le.fit_transform(X_train['Thickness'].values)

X_test['Size'] = class_le.fit_transform(X_test['Size'].values)
X_test['Number'] = class_le.fit_transform(X_test['Number'].values)
X_test['Thickness'] = class_le.fit_transform(X_test['Thickness'].values)

lung_cancer_le = LabelEncoder()
Y_train = lung_cancer_le.fit_transform(Y_train)

In [22]:
print(X_train)

      Size  Number  Thickness
0        0       1          1
1        2       2          1
2        4       0          1
3        0       3          2
4        4       3          0
...    ...     ...        ...
1973     0       0          2
1974     2       1          2
1975     0       2          1
1976     2       3          0
1977     1       2          0

[1978 rows x 3 columns]


In [23]:
# train a Gaussian Naive Bayes classifier on the training set
time_start_library = time.time()
nb = GaussianNB()
nb.fit(X_train, Y_train)

GaussianNB()

In [24]:
# predict the class labels of test set
Y_test_library = nb.predict(X_test)
time_end_library = time.time()

In [25]:
# print the predicted class labels
Y_test_arr_library = lung_cancer_le.inverse_transform(Y_test_library)
Y_test_df_library = pd.DataFrame(Y_test_arr_library, columns=['Lung Cancer'])

print(Y_test_library)
print(Y_test_arr_library)
print(Y_test_df_library)

[0 0 2 3 3 1 2 2 0 2]
['negative' 'negative' 'stage-2' 'stage-3' 'stage-3' 'stage-1' 'stage-2'
 'stage-2' 'negative' 'stage-2']
  Lung Cancer
0    negative
1    negative
2     stage-2
3     stage-3
4     stage-3
5     stage-1
6     stage-2
7     stage-2
8    negative
9     stage-2


In [28]:
print("Accuracy: ", accuracy_score(Y_test_arr_library, Y_test_arr))
print("Confusion Matrix: ", confusion_matrix(Y_test_arr_library, Y_test_arr))

Accuracy:  0.7
Confusion Matrix:  [[1 1 1 0]
 [0 1 0 0]
 [0 0 4 0]
 [0 0 1 1]]


In [27]:
# Running time
print(f"Thời gian chạy của thuật toán tự triển khai: \
      {time_end - time_start}")
print(f"Thời gian chạy của thuật toán thư viện: \
      {time_end_library - time_start_library}")

Thời gian chạy của thuật toán tự triển khai:       0.14911818504333496
Thời gian chạy của thuật toán thư viện:       0.03305172920227051
